In [1]:
!pip install opencv-python mediapipe numpy tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 50.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.1.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is inc

In [2]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [42]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

W0000 00:00:1733855939.999635    5630 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733855940.055935    5630 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [47]:
DATASET_PATH = "/kaggle/input/yoga-posture-dataset/"  # Replace with your dataset folder path
POSE_LABELS = ["Adho Mukha Svanasana", "Sivasana", "Setu Bandha Sarvangasana", "Balasana", "Salamba Sarvangasana","Virabhadrasana One","Virabhadrasana Two","Hanumanasana","Utkatasana","Vrksasana","Bakasana","Pincha Mayurasana","Paschimottanasana","Malasana","Trikonasana","Garudasana"]

In [44]:
data = []

In [45]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation
def extract_person(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Initialize Mediapipe Selfie Segmentation
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as selfie_segmentation:
        # Apply segmentation
        result = selfie_segmentation.process(image_rgb)
        
        # Create a binary mask
        mask = result.segmentation_mask > 0.5
        
        # Convert the mask to 3 channels
        mask_3d = np.dstack((mask, mask, mask))
        
        # Extract person by masking
        person_only = np.where(mask_3d, image, 0)  # Set background to black

        person_cnn_ready = cv2.resize(person_only, (224, 224))
        
        
        
        
        return person_cnn_ready

In [49]:
for pose_label in POSE_LABELS:
    pose_dir = os.path.join(DATASET_PATH, pose_label)
    for img_file in os.listdir(pose_dir):
        img_path = os.path.join(pose_dir, img_file)
        
        # Read the image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Failed to read {img_path}")
            continue
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process the image with Mediapipe
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            # Extract keypoints
            landmarks = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
            data.append([pose_label] + landmarks.tolist())

        else:
            print(f"No landmarks detected for {img_file} {pose_label}")

libpng warning: iCCP: known incorrect sRGB profile


No landmarks detected for File45.png Adho Mukha Svanasana
No landmarks detected for File43.png Adho Mukha Svanasana
No landmarks detected for File40.png Adho Mukha Svanasana
No landmarks detected for File68.png Adho Mukha Svanasana


libpng warning: cHRM: inconsistent chromaticities


No landmarks detected for File41.png Adho Mukha Svanasana
No landmarks detected for File33.png Adho Mukha Svanasana
No landmarks detected for File27.png Adho Mukha Svanasana


libpng warning: iCCP: known incorrect sRGB profile


No landmarks detected for File6.jpeg Sivasana
No landmarks detected for File2.png Sivasana
No landmarks detected for File9.jpeg Sivasana
No landmarks detected for File53.png Setu Bandha Sarvangasana
No landmarks detected for File12.png Setu Bandha Sarvangasana
No landmarks detected for File60.jpg Setu Bandha Sarvangasana
No landmarks detected for File49.jpg Setu Bandha Sarvangasana
No landmarks detected for File68.jpeg Setu Bandha Sarvangasana
No landmarks detected for File50.png Balasana
No landmarks detected for File75.jpeg Balasana
No landmarks detected for File73.jpeg Balasana
No landmarks detected for File71.png Balasana
No landmarks detected for File4.png Balasana
No landmarks detected for File22.png Balasana
No landmarks detected for File29.png Balasana
No landmarks detected for File67.png Balasana
No landmarks detected for File19.png Balasana
No landmarks detected for File77.jpeg Balasana
No landmarks detected for File25.png Balasana
No landmarks detected for File47.png Balasan

In [50]:
print(len(data))

899


In [51]:
columns = ["label"] + [f"kp_{i}" for i in range(len(data[0]) - 1)]
df = pd.DataFrame(data, columns=columns)
df.to_csv("/kaggle/working/yoga_pose_dataset_hindi.csv", index=False)
print("Keypoints dataset saved as yoga_pose_dataset_hindi.csv")

Keypoints dataset saved as yoga_pose_dataset_hindi.csv


In [52]:
df=pd.read_csv("/kaggle/working/yoga_pose_dataset_hindi.csv")
df

,label,kp_0,kp_1,kp_2,kp_3,kp_4,kp_5,kp_6,kp_7,kp_8,...,kp_122,kp_123,kp_124,kp_125,kp_126,kp_127,kp_128,kp_129,kp_130,kp_131
0,Adho Mukha Svanasana,0.574411,0.857551,-0.170344,0.999894,0.591441,0.873120,-0.140386,0.999928,0.591314,...,-0.363653,0.989478,0.135086,1.190710,0.012924,0.709866,0.097210,1.103287,-0.525023,0.988772
1,Adho Mukha Svanasana,0.475611,0.779400,-0.009473,0.999895,0.498640,0.765088,0.000471,0.999926,0.501930,...,0.045478,0.990265,0.197447,1.055179,0.271332,0.733142,0.167019,1.091692,-0.123786,0.989672
2,Adho Mukha Svanasana,0.642365,0.575218,-0.232704,0.991229,0.652018,0.583455,-0.289695,0.980736,0.657331,...,0.187522,0.990062,0.887097,0.989996,0.121519,0.758116,0.639090,1.160481,0.003206,0.990143
3,Adho Mukha Svanasana,0.591160,0.766361,-0.095115,0.992091,0.613317,0.750737,-0.069707,0.982647,0.615732,...,0.072176,0.990535,0.226271,0.921706,0.305065,0.771940,0.184570,0.964806,-0.165102,0.990566
4,Adho Mukha Svanasana,0.597835,0.664395,-0.102759,0.992867,0.612002,0.665718,-0.093990,0.984357,0.615508,...,0.044320,0.990819,0.306375,0.785221,0.367678,0.788215,0.060501,0.201757,-0.042143,0.990312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,Garudasana,0.489676,0.149583,-0.350364,0.913173,0.538343,0.129542,-0.453986,0.905653,0.548820,...,1.295999,0.561387,0.608267,1.006040,-0.183446,0.596816,0.646494,0.921103,1.332347,0.595358
895,Garudasana,0.431207,0.110012,0.138711,0.915965,0.305819,0.087942,0.144793,0.910139,0.198694,...,-0.551932,0.592104,0.187168,0.914089,-0.620453,0.625855,0.296571,0.927958,-0.747754,0.621591
896,Garudasana,0.450601,0.132711,-0.661518,0.924345,0.487520,0.113603,-0.745464,0.919108,0.493362,...,1.486805,0.619465,0.536439,0.934216,0.552201,0.657348,0.530747,0.987327,1.572088,0.645284
897,Garudasana,0.532836,0.178418,-0.249124,0.931238,0.494924,0.159973,-0.194860,0.926391,0.499138,...,-0.027417,0.649207,0.358651,0.958962,1.469819,0.661510,0.389217,0.964388,-0.004556,0.670024


In [53]:
import random
random.shuffle(data)

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [55]:
data = pd.read_csv("/kaggle/working/yoga_pose_dataset_hindi.csv")

In [56]:
X = data.drop("label", axis=1).values  # Keypoints
y = data["label"].values 

In [57]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [58]:
y_encoded

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8

In [59]:
y_onehot = np.eye(len(np.unique(y_encoded)))[y_encoded]

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.20, random_state=42)


In [66]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(y_onehot[0]), activation='softmax')  # Number of output classes
])

In [71]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/step - accuracy: 0.6117 - loss: 0.9450 - val_accuracy: 0.7639 - val_loss: 0.8537
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6844 - loss: 0.8391 - val_accuracy: 0.7569 - val_loss: 0.8453
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6986 - loss: 0.8728 - val_accuracy: 0.7292 - val_loss: 0.8365
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7162 - loss: 0.7831 - val_accuracy: 0.7361 - val_loss: 0.8304
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6561 - loss: 0.8655 - val_accuracy: 0.7222 - val_loss: 0.8293
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6962 - loss: 0.9226 - val_accuracy: 0.7361 - val_loss: 0.8331
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6950 - loss: 0.7579 - val_accuracy: 0.7431 - val_loss: 0.8402
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7239 - loss: 0.7272 - val_accuracy: 0.7500 -

In [73]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7700 - loss: 1.2099
Test Accuracy: 75.56%


In [86]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Build the enhanced model
model = Sequential([
    Dense(2048, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(128, activation='relu'),
    Dropout(0.3),
    
    Dense(len(y_onehot[0]), activation='softmax')  # Output layer with number of classes
])




In [87]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [88]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 494ms/step - accuracy: 0.0764 - loss: 3.4800 - val_accuracy: 0.1875 - val_loss: 2.6742
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3139 - loss: 2.1752 - val_accuracy: 0.2778 - val_loss: 2.5609
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4495 - loss: 1.7636 - val_accuracy: 0.3542 - val_loss: 2.3195
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4386 - loss: 1.6100 - val_accuracy: 0.3472 - val_loss: 2.2741
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5281 - loss: 1.5025 - val_accuracy: 0.3264 - val_loss: 2.2933
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5522 - loss: 1.3619 - val_accuracy: 0.3472 - val_loss: 2.3165
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6058 - loss: 1.1483 - val_accuracy: 0.3333 - val_loss: 2.2392
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6003 - loss: 1.1665 - val_accuracy: 0.2708 

In [89]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8242 - loss: 0.8653
Test Accuracy: 82.22%


In [93]:
model.save("/kaggle/working/yoga_pose_neural_network_model_hindi.h5")

In [90]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [91]:
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    use_label_encoder=False
)

xgb_model.fit(X_train, y_train)

# Evaluate the model
y_pred = xgb_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 72.78%
                          precision    recall  f1-score   support

    Adho Mukha Svanasana       0.79      0.79      0.79        14
                Bakasana       1.00      0.81      0.90        16
                Balasana       1.00      0.55      0.71        11
              Garudasana       1.00      0.78      0.88        18
            Hanumanasana       1.00      0.86      0.92         7
                Malasana       1.00      0.92      0.96        13
       Paschimottanasana       0.57      0.67      0.62         6
       Pincha Mayurasana       1.00      0.60      0.75         5
    Salamba Sarvangasana       0.91      0.83      0.87        12
Setu Bandha Sarvangasana       1.00      0.73      0.84        11
                Sivasana       0.00      0.00      0.00         7
             Trikonasana       1.00      0.75      0.86         4
              Utkatasana       0.88      0.70      0.78        10
      Virabhadrasana One       0.93      0.78      0.85   

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [92]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees in the forest
    max_depth=None,    # Maximum depth of the tree (None means no limit)
    random_state=42,   # Seed for reproducibility
    n_jobs=-1          # Use all available CPU cores
)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Test Accuracy: 72.22%

Classification Report:
                          precision    recall  f1-score   support

    Adho Mukha Svanasana       0.93      0.93      0.93        14
                Bakasana       1.00      0.81      0.90        16
                Balasana       1.00      0.36      0.53        11
              Garudasana       1.00      0.72      0.84        18
            Hanumanasana       0.88      1.00      0.93         7
                Malasana       1.00      1.00      1.00        13
       Paschimottanasana       1.00      0.67      0.80         6
       Pincha Mayurasana       1.00      0.60      0.75         5
    Salamba Sarvangasana       0.92      0.92      0.92        12
Setu Bandha Sarvangasana       1.00      0.82      0.90        11
                Sivasana       0.00      0.00      0.00         7
             Trikonasana       1.00      0.75      0.86         4
              Utkatasana       0.86      0.60      0.71        10
      Virabhadrasana One     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
